In [58]:
# cnn lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import MaxPooling1D
from keras.layers.convolutional import Conv1D

from matplotlib import pyplot

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_style("darkgrid")

import common_functions as cf

## Import Data

In [29]:
df_all = pd.read_pickle("merged_w_df.pkl")

In [30]:
df_all.shape

(128500, 15)

In [5]:
#np.asarray(df_all).reshape(2570,50,15)

In [7]:
# import pickle
# with open('merged_w_arr.pkl','rb') as f:
#     arr_all = pickle.load(f)
# arr_all.shape

(2570, 50, 16)

In [33]:
df_all.head(2)

,rotR_x,rotR_y,rotR_z,acc_x,acc_y,acc_z,or_x,or_y,or_z,grav_x,grav_y,grav_z,action,label,action_num
time,,,,,,,,,,,,,,,
0,-12.012195,5.243902,1.158537,0.098145,0.047607,0.029785,15,13,80,0.271240,-0.230225,0.934326,t_stand_left_DrinkBottle_10ml_1s_right,1,0
1,-23.780488,17.195122,3.658537,0.093994,0.043213,0.041748,15,14,80,0.259766,-0.249023,0.932861,t_stand_left_DrinkBottle_10ml_1s_right,1,0


# Feature engineering (basic)

In [31]:
df_fe = cf.fe_basic_features(df_all)

In [34]:
df_fe.head(2)

,accel_norm,rotR_x,rotR_y,rotR_z,acc_x,acc_y,acc_z,or_x,or_y,or_z,grav_x,grav_y,grav_z,action,label,action_num
time,,,,,,,,,,,,,,,,
0,1.048477,-12.012195,5.243902,1.158537,0.098145,0.047607,0.029785,15,13,80,0.271240,-0.230225,0.934326,t_stand_left_DrinkBottle_10ml_1s_right,1,0
1,1.057056,-23.780488,17.195122,3.658537,0.093994,0.043213,0.041748,15,14,80,0.259766,-0.249023,0.932861,t_stand_left_DrinkBottle_10ml_1s_right,1,0


In [32]:
df_fe.shape

(128500, 16)

## Data Splitting(Test/train)

In [35]:
train_df,test_df = cf.data_split_TrainTest(df_fe)

In [36]:
train_df.shape,test_df.shape

((72150, 16), (56350, 16))

## Preprocess (Normalizing)

In [37]:
from sklearn.preprocessing import StandardScaler
last_feat_column = df_fe.columns[-4]

for act_num in train_df.action_num.unique():
    std = StandardScaler().set_output(transform='pandas')
    
    train_df_w_act_num = train_df.loc[train_df.action_num == act_num,:last_feat_column].copy()
    std.fit(train_df_w_act_num)
    train_df.loc[train_df.action_num == act_num,:last_feat_column] = std.transform(train_df_w_act_num)


for act_num in test_df.action_num.unique():
    std = StandardScaler().set_output(transform='pandas')
    
    test_df_w_act_num = test_df.loc[test_df.action_num == act_num,:last_feat_column].copy()
    std.fit(test_df_w_act_num)
    test_df.loc[test_df.action_num == act_num,:last_feat_column] = std.transform(test_df_w_act_num)

In [38]:
train_df.shape,test_df.shape

((72150, 16), (56350, 16))

In [11]:
#X_train_roll = cf.pre_normalize_tanh(X_train_roll)
#X_test_roll = cf.pre_normalize_tanh(X_test_roll)

## Modeling

### Split Data (Feature/Label)

In [49]:
def load_dataset():
    X_train,y_train = cf.data_split_FeatLabel(train_df)
    X_test,y_test = cf.data_split_FeatLabel(test_df)
   
    X_train = np.asarray(X_train).reshape(int(X_train.shape[0]/cf.SEQUENCE_SIZE) , cf.SEQUENCE_SIZE, X_train.shape[1])
    y_train = np.asarray(y_train).reshape(int(y_train.shape[0]/cf.SEQUENCE_SIZE) , cf.SEQUENCE_SIZE)

    X_test = np.asarray(X_test).reshape(int(X_test.shape[0]/cf.SEQUENCE_SIZE) , cf.SEQUENCE_SIZE, X_test.shape[1])
    y_test = np.asarray(y_test).reshape(int(y_test.shape[0]/cf.SEQUENCE_SIZE) , cf.SEQUENCE_SIZE)

    return  X_train,y_train,X_test,y_test

### Neurel Network

In [48]:
# load data
trainX, trainy, testX, testy = load_dataset()

X_train   (72150, 13)
X_train   (1443, 50, 13)
y_train   (72150,)
y_train   (1443, 50)


In [62]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

# reshape data into time steps of sub-sequences
n_steps = 2
n_length = int(cf.SEQUENCE_SIZE/n_steps) 

trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))

# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
model.fit(trainX, trainy, epochs=50, batch_size=25, verbose="0")
# evaluate model
_, accuracy = model.evaluate(testX, testy, batch_size=25, verbose="0")

print('Accuracy: %.3f' % (accuracy*100))

Accuracy: 0.799


In [59]:
# # fit and evaluate a model
# def evaluate_model(trainX, trainy, testX, testy):
# 	# define model
# 	verbose, epochs, batch_size = 0, 50, 25
# 	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
# 	# reshape data into time steps of sub-sequences
# 	n_steps = 2
# 	n_length = int(cf.SEQUENCE_SIZE/n_steps) #25
# 	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
# 	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))

# 	# define model
# 	model = Sequential()
# 	model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
# 	model.add(TimeDistributed(Conv1D(filters=32, kernel_size=3, activation='relu')))
# 	model.add(TimeDistributed(Dropout(0.5)))
# 	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
# 	model.add(TimeDistributed(Flatten()))
# 	model.add(LSTM(100))
# 	model.add(Dropout(0.5))
# 	model.add(Dense(100, activation='relu'))
# 	model.add(Dense(n_outputs, activation='sigmoid'))
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	# fit network
# 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
# 	# evaluate model
# 	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
# 	return accuracy

In [41]:
# # summarize scores
# def summarize_results(scores):
# 	print(scores)
# 	m, s = mean(scores), std(scores)
# 	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# # run an experiment
# def run_experiment(repeats=10):
# 	# load data
# 	trainX, trainy, testX, testy = load_dataset()
# 	# repeat experiment
# 	scores = list()
# 	for r in range(repeats):
# 		score = evaluate_model(trainX, trainy, testX, testy)
# 		score = score * 100.0
# 		print('>#%d: %.3f' % (r+1, score))
# 		scores.append(score)
# 	# summarize results
# 	summarize_results(scores)

In [ ]:
# run the experiment
# run_experiment(1)